In [28]:
import numpy as np
import sounddevice as sd
import matplotlib.pyplot as plt
from scipy.signal import spectrogram
from scipy.fftpack import fft
from pynput import keyboard

# Constants
sampling_rate = 44100
duration = 1.0  # seconds

# Create dictionary for storing generated sounds
sounds = {}

# Note frequencies
note_frequencies = {
    'C4': 261.63,
    'D4': 293.66,
    'E4': 329.63,
    'F4': 349.23,
    'G4': 392.00,
    'A4': 440.00,
    'B4': 493.88,
    'C5': 523.25,
    'D5': 587.33,
    'E5': 659.25,
    'F5': 698.46,
    'G5': 783.99,
}


In [29]:
# Define the Karplus-Strong Algorithm
def karplus_strong(frequency, duration, sampling_rate=44100, decay=0.995):
    N = int(sampling_rate / frequency)
    noise = np.random.uniform(-1, 1, N)
    buffer = np.copy(noise)
    output = []
    for i in range(int(sampling_rate * duration)):
        output.append(buffer[0])
        avg = decay * 0.5 * (buffer[0] + buffer[1])
        buffer[:-1] = buffer[1:]
        buffer[-1] = avg
    return np.array(output)

# Function to play sound
def play_sound(sound, sampling_rate):
    sd.play(sound, samplerate=sampling_rate)
    sd.wait()

# Function to plot waveform
def plot_waveform(signal, title):
    plt.figure(figsize=(10, 4))
    plt.plot(signal)
    plt.title(f"Waveform of {title}")
    plt.xlabel("Sample")
    plt.ylabel("Amplitude")
    plt.show()

# Function to plot spectrogram
def plot_spectrogram(signal, sampling_rate, title):
    f, t, Sxx = spectrogram(signal, sampling_rate)
    plt.figure(figsize=(10, 4))
    plt.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.title(f'Spectrogram of {title}')
    plt.colorbar(label='Intensity [dB]')
    plt.show()

# Function to plot FFT
def plot_fft(signal, sampling_rate, title):
    N = len(signal)  # Number of samples in the signal
    T = 1.0 / sampling_rate  # Sampling interval
    yf = fft(signal)  # Perform FFT
    xf = np.linspace(0.0, 1.0 / (2.0 * T), N // 2)  # Frequency axis for plotting

    plt.figure(figsize=(10, 4))
    plt.plot(xf, 2.0 / N * np.abs(yf[:N // 2]))  # Plot FFT
    plt.grid()
    plt.title(f"FFT of {title}")
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Amplitude")
    plt.show()


In [30]:
# Store played chords
played_chords = []

# Define melodies
happy_birthday_melody = [
    ('G4', []), ('G4', []), ('A4', []), ('G4', []), ('C5', []), ('B4', []),
    ('G4', []), ('G4', []), ('A4', []), ('G4', []), ('D5', []), ('C5', []),
    ('G4', []), ('G4', []), ('G5', []), ('E5', []), ('C5', []), ('B4', []), ('A4', []),
    ('F5', []), ('F5', []), ('E5', []), ('C5', []), ('D5', []), ('C5', [])
]

twinkle_twinkle_melody = [
    ('C4', []), ('C4', []), ('G4', []), ('G4', []), ('A4', []), ('A4', []), ('G4', []),
    ('F4', []), ('F4', []), ('E4', []), ('E4', []), ('D4', []), ('D4', []), ('C4', []),
    ('G4', []), ('G4', []), ('F4', []), ('F4', []), ('E4', []), ('E4', []), ('D4', []),
    ('G4', []), ('G4', []), ('F4', []), ('F4', []), ('E4', []), ('E4', []), ('D4', []),
    ('C4', []), ('C4', []), ('G4', []), ('G4', []), ('A4', []), ('A4', []), ('G4', []),
    ('F4', []), ('F4', []), ('E4', []), ('E4', []), ('D4', []), ('D4', []), ('C4', [])
]


In [31]:
def create_combined_track(melody, duration=0.5, sampling_rate=44100):
    track = np.array([])
    for note, _ in melody:
        if note in note_frequencies:
            sound = karplus_strong(note_frequencies[note], duration, sampling_rate)
            track = np.concatenate((track, sound))
    return track


In [32]:
def play_and_plot_melody(melody, title):
    combined_sound = create_combined_track(melody)
    play_sound(combined_sound, sampling_rate)
    plot_waveform(combined_sound, title)
    plot_spectrogram(combined_sound, sampling_rate, title)
    plot_fft(combined_sound, sampling_rate, title)


In [33]:
def on_press(key):
    try:
        if key.char in 'cdefgab':
            note = key.char.upper() + '4'  # Assume 4th octave
            if note in note_frequencies:
                sound = karplus_strong(note_frequencies[note], 0.5, sampling_rate)
                play_sound(sound, sampling_rate)
                plot_waveform(sound, note)
                plot_spectrogram(sound, sampling_rate, note)
                plot_fft(sound, sampling_rate, note)
                played_chords.append((note, []))
        elif key.char == 'x':
            play_and_plot_melody(happy_birthday_melody, 'Happy Birthday')
        elif key.char == 'y':
            play_and_plot_melody(twinkle_twinkle_melody, 'Twinkle Twinkle Little Star')
        elif key.char == 'z':
            all_notes = ['C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5', 'D5', 'E5', 'F5', 'G5']
            combined_melody = [(note, []) for note in all_notes]
            play_and_plot_melody(combined_melody, 'All Combined Notes from C4 to G5')
        elif key.char == 'w':
            if played_chords:
                play_and_plot_melody(played_chords, 'All Played Notes')
            else:
                print("No chords have been played yet.")
        elif key.char == 'q':
            return False
    except AttributeError:
        print(f"Special key {key} pressed.")
def main():
    print("Press keys 'c', 'd', 'e', 'f', 'g', 'a', 'b' for corresponding notes,'x' for Happy Birthday melody, 'y' for Twinkle Twinkle melody, 'z' for sequential chords, 'w' for played chords, or 'q' to quit.")
    with keyboard.Listener(on_press=on_press) as listener:
        listener.join()

if __name__ == "__main__":
    main

